# Pluggable Python


In [47]:
import itertools
from dataclasses import dataclass
from typing import runtime_checkable, Protocol, Callable
from abc import abstractmethod
from setuptools import setup



## The Template Method Pattern


In [6]:
# Consider the first implementation of a pizza maker function, as follows:

@dataclass
class PizzaCreationFunctions:
    prepare_ingredients: Callable
    add_pre_bake_toppings: Callable
    add_post_bake_toppings: Callable

def roll_out_pizza_base():
    return

def bake_pizza():
    return

def create_pizza(pizza_creation_functions: PizzaCreationFunctions):
    pizza_creation_functions.prepare_ingredients()
    roll_out_pizza_base()
    pizza_creation_functions.add_pre_bake_toppings()
    bake_pizza()
    pizza_creation_functions.add_post_bake_toppings()


In [7]:
# Now, to create a specific pizza type, we you just pass in the custom functions:

def mix_zaatar(): 
    return

def add_meat_and_halloumi(): 
    return

def drizzle_olive_oil(): 
    return

pizza_creation_functions = PizzaCreationFunctions(
    prepare_ingredients=mix_zaatar,
    add_pre_bake_toppings=add_meat_and_halloumi,
    add_post_bake_toppings=drizzle_olive_oil
)

create_pizza(pizza_creation_functions)


## The Strategy Pattern


In [18]:
# Suppose the Ultimate Kitchen Assistant has numbered bins to put ingredients in. 
# For example, code specialized in 'Tex-Mex' cuisine will be similar to this:

@dataclass
class TexMexIngredients:
    corn_tortilla_bin: int
    flour_tortilla_bin: int
    salsa_bin: int
    ground_beef_bin: int
    shredded_cheese_bin: int

def get_available_ingredients(ingredient):
    return
    
def prepare_tex_mex_dish(tex_mex_recipe_maker: Callable):
    tex_mex_ingredients = get_available_ingredients("Tex-Mex")
    dish = tex_mex_recipe_maker(tex_mex_ingredients)
    serve(dish)


In [29]:
# And a function that prepares a specific Tex-Mex based dish will be something like this:

"""
import tex_mex_module as tmm
"""

def make_soft_taco(ingredients: TexMexIngredients) -> tmm.Dish:
    tortilla = tmm.get_ingredient_from_bin(ingredients.flour_tortilla_bin)
    beef = tmm.get_ingredient_from_bin(ingredients.ground_beef_bin)
    dish = tmm.get_plate()
    dish.lay_on_dish(tortilla)
    tmm.season(beef, tmm.CHILE_POWDER_BLEND)

"""
prepare_tex_mex_dish(make_soft_taco)
"""


'\nprepare_tex_mex_dish(make_soft_taco)\n'

In [28]:
# If we want to provide support for a different dish, we just write a new function:

def make_chimichanga(ingredients: TexMexIngredients):
    # ... snip
    return

## Plug-in Architectures


In [43]:
# Before continuing, install and import the 'stevedore' library, as follows

"""
!pip install stevedore
"""
import stevedore


In [38]:
# Consider the following Ultimate Kitchen Assistant abstract class:

class Ingredient:
    pass

class Recipe:
    pass

class Amount:
    pass

class Dish:
    pass


@runtime_checkable
class UltimateKitchenAssistantModule(Protocol):
    ingredients: list[Ingredient]
    
    @abstractmethod
    def get_recipes() -> list[Recipe]:
        raise NotImplementedError
    
    @abstractmethod
    def prepare_dish(
        inventory: dict[Ingredient, Amount],
        recipe: Recipe
    ) -> Dish:
        raise NotImplementedError


In [39]:
# To create a new plug-in, we just create a class that inherits from the base class:

class PastaModule(UltimateKitchenAssistantModule):
    def __init__(self):
        self.ingredients = ["Linguine", "Spaghetti" ]
    
    def get_recipes(self) -> list[Recipe]:
        # ... snip returning all possible recipes ...
        return 

    def prepare_dish(
        self, 
        inventory: dict[Ingredient, Amount],
        recipe: Recipe
    ) -> Dish:
        # interact with Ultimate Kitchen Assistant to make recipe
        return 


In [45]:
# Once the plug-in ha been created,  we need to register plug-ins 
# with the help of setuptools and setup.py, as follows:

"""
setup(
    name='ultimate_kitchen_assistant',
    version='1.0',
    #.... snip ....
    entry_points={
        'ultimate_kitchen_assistant.recipe_maker': [
            'pasta_maker = ultimate_kitchen_assistant.pasta_maker:PastaModule',
            'tex_mex = ultimate_kitchen_assistant.tex_mex:TexMexModule'
        ],
    },
)
"""

"\nsetup(\n    name='ultimate_kitchen_assistant',\n    version='1.0',\n    #.... snip ....\n    entry_points={\n        'ultimate_kitchen_assistant.recipe_maker': [\n            'pasta_maker = ultimate_kitchen_assistant.pasta_maker:PastaModule',\n            'tex_mex = ultimate_kitchen_assistant.tex_mex:TexMexModule'\n        ],\n    },\n)\n"

In [48]:
# Once the plug-ins are registered, we can use 'stevedore' to load them at runtime. 
# For example, we can collect all recipes from all plugins as follows:

class Recipe:
    pass


def get_all_recipes() -> list[Recipe]:
    mgr = stevedore.extension.ExtensionManager(
        namespace='ultimate_kitchen_assistant.recipe_maker',
        invoke_on_load=True,
    )

    return list(itertools.chain(
        mgr.map(lambda extension: extension.obj.get_recipes())
    ))


In [49]:
# Suppose an user wants to make something from the pasta maker. 
# In that case, the calling code should ask for a plug-in named pasta_maker,
# which can be done with stevedore.driver.DriverManager, as follows:

def get_inventory():
    return

def make_dish(recipe: Recipe, module_name: str) -> Dish:
    mgr = stevedore.driver.DriverManager(
        namespace='ultimate_kitchen_assistant.recipe_maker',
        name=module_name,
        invoke_on_load=True,
    )
    
    return mgr.driver.prepare_dish(get_inventory(), recipe)
